# Registration covariance

Analysis of the behavior of two registration algorithms: ICP and NDP.
I will use pandas for this.

The purpose of this document is to analyse data collected with the registration-covariance program.
This dataset contains a lot of pointcloud registration runs, so we can analyze the behavior of these algorithms.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pathlib

First let's load the data.

In [ ]:
frames = []

path = pathlib.Path('/home/david/dataset/recov')
#files = list(path.iterdir())
#files = ['/home/david/dataset/recov/2017-05-17-apartment.csv',
#        '/home/david/dataset/recov/2017-05-17-gazebo-summer.csv',
#        '/home/david/dataset/recov/2017-05-17-gazebo-winter.csv',
#         '/home/david/dataset/recov/2017-05-17-hauptgebaude.csv',
#         '/home/david/dataset/recov/2017-05-17-plain.csv',
#         '/home/david/dataset/recov/2017-05-17-wood-autumn.csv',
#        '/home/david/dataset/recov/2017-05-17-wood-summer.csv']

files = ['/home/dlandry/dataset/recov/apartment.csv']

for file in files:
    entry = pathlib.Path(file)
    if entry.suffix == '.csv' and entry.is_file():
        frames.append(pd.read_csv(entry))
    
dataset = pd.concat(frames)
dataset.index = range(len(dataset))

len(dataset)

In [ ]:
dataset[0:3]

Fun fact: we ran a lot of registrations to get this dataset.

In [ ]:
dataset['n_samples'].sum()

## ICP Algorithm

To begin, let's analyze the behavior of ICP when we play with various variables.

In [ ]:
icp_dataset = dataset[dataset['algorithm'] == 'icp']

### Distribution of errors

In [ ]:
fig, [[axx, axy], [axz, axt]] = plt.subplots(2,2, sharey=True, figsize=(20,10))

dataset.hist(column='bias_x', ax=axx, bins=100)
dataset.hist(column='bias_y', ax=axy, bins=100)
dataset.hist(column='bias_z', ax=axz, bins=100)
dataset.hist(column='bias_theta', ax=axt, bins=100)

plt.show()

In [ ]:
dataset[dataset['dataset'] != 'apartment'].hist(column='bias_y', by='algorithm', bins=50)
dataset.hist(column='bias_y', by='dataset', bins=50)
plt.show()

In [ ]:
dataset.std()

In [ ]:
dataset.boxplot(column='var_x', by='algorithm')
plt.show()

In [ ]:
icp_dataset.boxplot(column='gt_mahalanobis_distance')
plt.show()

In [ ]:
dataset.boxplot(column='bias_norm', by='algorithm')
plt.show()

In [ ]:
icp_dataset[icp_dataset['reading'] - icp_dataset['reference'] < 7].boxplot(column='bias_norm', by='dataset')
plt.show()

In [ ]:
icp_dataset.boxplot(column='rotation_mean_gt', by='dataset')
plt.show()

### Effect of the distance between the input scans

This first plot shows how far the result is from the ground truth as the difficulty of the registration increases. 
By result we mean the distribution of transformations output by the registration algorithm.
By increased difficulty we mean that the ground truth distance between the two scans to register is larger.

In [ ]:
icp_dataset.plot(kind='scatter', x='dist_between_scans', y='gt_mahalanobis_distance', s=1, figsize=(10,5))
plt.show()

In [ ]:
icp_dataset.plot(kind='scatter', x='dist_between_scans', y='translation_mean_gt', s=1, figsize=(10,5))
plt.show()

In [ ]:
fig, ax = plt.subplots()
icp_dataset[icp_dataset['dataset'] == 'coop'].plot(kind='scatter', x='dist_between_scans', y='translation_mean_gt', s=1, figsize=(10,5), ax=ax, color='red')
icp_dataset[icp_dataset['dataset'] == 'arla'].plot(kind='scatter', x='dist_between_scans', y='translation_mean_gt', s=1, figsize=(10,5), ax=ax, color='blue')
plt.show()

In [ ]:
fig, ax = plt.subplots()
icp_dataset[icp_dataset['dataset'] == 'coop'].plot(kind='scatter', x='dist_between_scans', y='rotation_mean_gt', s=1, figsize=(10,5), ax=ax, color='red')
icp_dataset[icp_dataset['dataset'] == 'arla'].plot(kind='scatter', x='dist_between_scans', y='rotation_mean_gt', s=1, figsize=(10,5), ax=ax, color='blue')
plt.show()

This next figure represents the growth of the covariance as the difficulty of the registration increases.
In this dataset it seems that it is safe to do registration up until 20-30cm of distance between both clouds, after that you become less certain about the result.

In [ ]:
icp_dataset['cov_norm'].max()

icp_dataset.plot(kind='scatter', x='dist_between_scans', y='cov_norm', s=1, figsize=(10,5), ylim=[0.0, 0.0002])
plt.show()

In [ ]:
ylim = (0., 0.5)
icp_dataset.plot(kind='scatter', x='cov_norm', y='gt_mahalanobis_distance', s=1, figsize=(10,5))
plt.show()

### Effect of the quality of the initial estimate

The results show that icp is indeed less reliable when the scans are further apart. 
But could it be because the quality of our initial estimates degraded?
Let's do some computations to see if it's the case.
First we augment our dataset with the disparity data.

Let's plot the closeness of the groud thruth with respect to the disparity of the estimate.

In [ ]:
icp_dataset = dataset[dataset['algorithm'] == 'icp']
icp_dataset.plot(kind='scatter', x='disparity_gt_estimate', y='gt_mahalanobis_distance', s=1, figsize=(10,5))
plt.show()

And now the size of the covariance with respect to the disparity of the estimate.

In [ ]:
ylim = (icp_dataset['cov_norm'].min(), icp_dataset['cov_norm'].max())
icp_dataset.plot(kind='scatter', x='disparity_gt_estimate', y='cov_norm', s=1, figsize=(10,5), ylim=ylim)
plt.show()

Let's see how the disparity and the distance between scans are corellated. If they are very heavily correlated, the last section doesn't tell us much.

In [ ]:
ylim = (icp_dataset['disparity_gt_estimate'].min(), icp_dataset['disparity_gt_estimate'].max())
icp_dataset.plot(kind='scatter', x='dist_between_scans', y='disparity_gt_estimate', s=1, figsize=(10,5), ylim=ylim)
plt.show()

We also took note of the covariance of the noise that we artificially induced in the initial estimate.
That's another quality metric of the initial estimate.
Let's see how it affects the result.

In [ ]:
ax = icp_dataset.boxplot(column='gt_mahalanobis_distance', by='initial_estimate_covariance')
plt.show()

In [ ]:
ax = icp_dataset.boxplot(column='translation_mean_gt', by='initial_estimate_covariance')
plt.show()

Interestingly, it does not seem to affect the general quality of the estimates, but it does seem to affect the number of outliers. 
We have to note though that outliers in terms of quantiles does not mean outliers in terms of result; a mahalanobis distance of 0.003, even though it is an outlier of the distribution of results, is still a very low error.

In [ ]:
icp_dataset[icp_dataset['gt_mahalanobis_distance'] > 0.003].groupby('initial_estimate_covariance').count()

As a proof that these outliers are still satisfactory results, let's see what their human measurements counterpart are.

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim([0.0, 0.001])
icp_dataset.plot(kind='scatter', x='gt_mahalanobis_distance', y='translation_mean_gt', s=1, ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim([0.0, 0.01])
icp_dataset.plot(kind='scatter', x='gt_mahalanobis_distance', y='rotation_mean_gt', s=1, ax=ax)
plt.show()

### Computation time

Let's see how our difficulty metrics affect the computation time, since ICP is an iterative algorithm.

In [ ]:
icp_dataset.plot(kind='scatter', x='dist_between_scans', y='avg_cpu_time', s=1, figsize=(10,5))
plt.show()

In [ ]:
icp_dataset.plot(kind='scatter', x='disparity_gt_estimate', y='avg_cpu_time', s=1, figsize=(10,5))
plt.show()

In [ ]:
fig, ax = plt.subplots()
for i, group in icp_dataset.groupby('dataset'):
    ax.scatter(x=group['disparity_gt_estimate'], y=group['avg_cpu_time'], s=1)

plt.show()

### Covariance and error

In [ ]:
icp_dataset.plot(kind='scatter', x='gt_mahalanobis_distance', y='cov_norm', s=1, xlim=[0.0, 0.0005], ylim=[0.0, 0.001])
plt.show()

## The NDT Algorithm

Let's produce similar graphs for NDT.

In [ ]:
ndt_dataset = dataset.loc[dataset['algorithm'] == 'ndt', :]

In [ ]:
ax = ndt_dataset.boxplot(column='bias_norm')
ax.set_ylim([0., 0.5])
plt.show()

In [ ]:
ax = ndt_dataset.boxplot(column='rotation_mean_gt')
ax.set_ylim([0., 0.5])
plt.show()

In [ ]:
ndt_dataset.plot(kind='scatter', x='dist_between_scans', y='gt_mahalanobis_distance', s=1, figsize=(10,5), ylim=[0., 2.])
plt.show()

In [ ]:
ndt_dataset.plot(kind='scatter', x='dist_between_scans', y='bias_norm', s=1, figsize=(10,5), ylim=[0., 1.])
plt.show()

In [ ]:
ndt_dataset.plot(kind='scatter', x='dist_between_scans', y='rotation_mean_gt', s=1, figsize=(10,5), ylim=[0., 0.4])
plt.show()

In [ ]:
ndt_dataset.plot(kind='scatter', x='dist_between_scans', y='gt_mahalanobis_distance', s=1, figsize=(10,5), ylim=[0., 2.])
plt.show()

In [ ]:
ndt_dataset.plot(kind='scatter', x='dist_between_scans', y='cov_norm', s=1, figsize=(10,5), ylim=[0.0, 1.0])
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.set_ylim([0., 1.0])
ndt_dataset[ndt_dataset['dataset'] == 'coop'].plot(kind='scatter', x='dist_between_scans', y='bias_norm', s=1, figsize=(10,5), ax=ax, color='red')
ndt_dataset[ndt_dataset['dataset'] == 'arla'].plot(kind='scatter', x='dist_between_scans', y='bias_norm', s=1, figsize=(10,5), ax=ax, color='blue')
plt.show()

The next one is very interesting.
Because of the grid, NDT seems only to be able to output rotation by discrete steps.

In [ ]:
fig, ax = plt.subplots()
ndt_dataset[ndt_dataset['dataset'] == 'coop'].plot(kind='scatter', x='dist_between_scans', y='rotation_mean_gt', s=1, figsize=(10,5), ax=ax, color='red')
ndt_dataset[ndt_dataset['dataset'] == 'arla'].plot(kind='scatter', x='dist_between_scans', y='rotation_mean_gt', s=1, figsize=(10,5), ax=ax, color='blue')
plt.show()

In [ ]:
ylim = (ndt_dataset['cov_norm'].min(), 1)
ndt_dataset.plot(kind='scatter', x='disparity_gt_estimate', y='cov_norm', s=1, figsize=(10,5), ylim=ylim)
plt.show()

### Impact of the resolution

The choice of resolutions for the NDT grids is a difficult parameter for the NDT algorithm.
First we parse the registration config to extract the resolution used.

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
dataset.boxplot(column='gt_mahalanobis_distance', by='config', figsize=(10,5), ax=ax)
ax.set_ylim(0., 2.)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
dataset.boxplot(column='cov_norm', by='config',ax=ax)
ax.set_ylim([0.0, 5.0])
plt.show()

We could also have a look at the change in compute time as the resolution increases.

In [ ]:
dataset.boxplot(column='avg_cpu_time', by='config', figsize=(10,5))
plt.show()

In [ ]:
ndt_dataset.plot(kind='scatter', x='avg_cpu_time', y='gt_mahalanobis_distance', s=1, ylim=[0., 1.])
plt.show()

## Correlation matrices

In [ ]:
variables_of_interest = ['bias_norm', 'rotation_mean_gt', 'gt_mahalanobis_distance', 'cov_norm', 'dist_between_scans', 'avg_cpu_time', 'censi_cov_norm', 'translation_gt', 'rotation_gt']

### For all data

In [ ]:
dataset[variables_of_interest].corr('spearman')

### For ICP

In [ ]:
icp_dataset[variables_of_interest].corr('spearman')

### For NDT

In [ ]:
ndt_dataset[variables_of_interest].corr('spearman')

### For good NDT

In [ ]:
good_ndt = ndt_dataset[ndt_dataset['config'] == '{"resolutions": [0.5]}']
good_ndt[['gt_mahalanobis_distance', 'cov_norm', 'dist_between_scans', 'avg_cpu_time', 'censi_cov_norm']].corr('spearman')

In [ ]:
len(good_ndt)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax.set_title('Censi covariance estimate with respect to the sampled covariance')


ax = good_ndt.plot(kind='scatter', x='cov_norm', y='censi_cov_norm', s=1, figsize=(10,5), ax=ax, xlim=[0,2], ylim=[0,0.000005])

ax.set_xlabel('Matrix norm of the sampled covariance')
ax.set_ylabel('Matrix norm of the Censi covariance estimate')
plt.show()

## Comparisons

Here we see that ndt does not really compare to ICP in terms of precision, even at the best resolution.

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
                       
best_ndt = ndt_dataset[ndt_dataset['ndt_resolutions'] == '[0.5]']

ax.scatter(ndt_dataset['dist_between_scans'], ndt_dataset['gt_mahalanobis_distance'], s=1)
ax.scatter(icp_dataset['dist_between_scans'], icp_dataset['gt_mahalanobis_distance'], s=1)
ax.set_ylim([0.0, 0.05])

plt.show()

In [ ]:
dataset['gt_mahalanobis_distance'].plot(kind='box')
plt.show()

In [ ]:
ax = dataset.boxplot(column='gt_mahalanobis_distance', by='algorithm', figsize=(10, 6))
ax.set_ylim([0., 0.2])
plt.show()

In [ ]:
ax = dataset.boxplot(column='bias_norm', by='algorithm', figsize=(10, 6))
ax.set_ylim([0., 0.5])
plt.show()

In [ ]:
ax = dataset.boxplot(column='rotation_mean_gt', by='algorithm', figsize=(10, 6))
ax.set_ylim([0., 0.3])
plt.show()

In [ ]:
ax = dataset.boxplot(column='cov_norm', by='algorithm', figsize=(10, 6))
ax.set_ylim([0., .5])
plt.show()

In [ ]:
fig, ax = plt.subplots()
groups = dataset.sort_values('dist_between_scans').groupby('algorithm')

for name, group in groups:
    ax.scatter(group['dist_between_scans'], group['gt_mahalanobis_distance'], s=1.0)

ax.set_ylim([0, 1.0])
plt.show()

In [ ]:
dataset.groupby('algorithm').mean()

In [ ]:
ndt_dataset[ndt_dataset['cov_norm'] > 15]

In [ ]:
dataset.plot(kind='scatter', x='dist_between_scans', y='disparity_gt_estimate', s=1.)
plt.show()

In [ ]:
ax = dataset.boxplot(column='translation_mean_gt', by='algorithm')
ax.set_ylim([0., 1.0])
plt.show()

In [ ]:
ax = dataset.boxplot(column='rotation_mean_gt', by='algorithm')
ax.set_ylim([0., 0.2])
plt.show()

In [ ]:
icp_dataset.plot(kind='scatter', x='dist_between_scans', y='lie_norm_mean_gt', s=1)
plt.show()

### Number of outliers

We try to assess the robustness of each algorithm by counting the number of outliers in the error function.
A large number of outliers in the error indicate that some results are much further than usual from the ground truth.
It's safe to assume the registration failed there.

In [ ]:
ndt_Q1 = ndt_dataset['gt_mahalanobis_distance'].quantile(0.25)
ndt_Q3 = ndt_dataset['gt_mahalanobis_distance'].quantile(0.75)
ndt_IQ = ndt_Q3 - ndt_Q1
ndt_threshold = ndt_Q3 + 3. * ndt_IQ
ndt_dataset['gt_mahalanobis_distance'].where(ndt_dataset['gt_mahalanobis_distance'] > ndt_threshold).count() / ndt_dataset['gt_mahalanobis_distance'].count()

In [ ]:
icp_Q1 = icp_dataset['gt_mahalanobis_distance'].quantile(0.25)
icp_Q3 = icp_dataset['gt_mahalanobis_distance'].quantile(0.75)
icp_IQ = icp_Q3 - icp_Q1
icp_threshold = icp_Q3 + 3. * icp_IQ
icp_dataset['gt_mahalanobis_distance'].where(icp_dataset['gt_mahalanobis_distance'] > icp_threshold).count() / icp_dataset['gt_mahalanobis_distance'].count()

More classicaly we can set a fixed threshold on the error, and count how many registration runs ended up further than 

In [ ]:
dataset[dataset['bias_norm'] > 0.3].sort_values('dist_between_scans')

In [ ]:
for i, group in dataset.groupby(dataset['reading']-dataset['reference']):
    group.boxplot('bias_norm')
    
plt.show()

In [ ]:
dataset[dataset['dataset'] == 'coop'].plot(kind='scatter', x='reference', y='translation_mean_gt', s=1., ylim=[0., 1.])
plt.show()

In [ ]:
dataset[dataset['dataset'] == 'arla'].plot(kind='scatter', x='reference', y='translation_mean_gt', s=1., ylim=[0., 1.])
plt.show()

In [ ]:
dataset[dataset['dataset'] == 'arla'].plot(kind='scatter', x='reference', y='disparity_gt_estimate', s=1., ylim=[0., 0.5])
plt.show()

In [ ]:
dataset[dataset['dataset'] == 'coop'].plot(kind='scatter', x='reference', y='disparity_gt_estimate', s=1., ylim=[0., 0.1])
plt.show()

In [ ]:
dataset.plot(kind='scatter', x='disparity_gt_estimate', y='bias_norm', s=1, ylim=[0., 1.0])
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.set_ylim([0., 1.0])
colors = {'icp': 'red', 'ndt': 'blue'}

for name, group in dataset.groupby('algorithm'):
    ax.scatter(group['disparity_gt_estimate'], group['bias_norm'], s=1.)
    group.plot(kind='scatter', x='disparity_gt_estimate', y='bias_norm', ax=ax, s=1, color=colors[name])
    
plt.show()

In [ ]:
dataset[dataset['bias_norm'] > 10]

In [ ]:
icp_dataset.plot(kind='scatter', x='translation_gt', y='bias_norm', s=1)
plt.show()

In [ ]:
easy_matches = dataset[dataset['reading'] - dataset['reference'] == 4]
easy_matches.boxplot(column='bias_norm', by='algorithm')
plt.show()

In [ ]:
easy_matches[easy_matches['bias_norm'] > 0.1]